<a href="https://colab.research.google.com/github/cha0stooo/f16flyingdatasets/blob/master/main_net.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
#用自己的网络，包含bn，但是采用tf进行自动微分
import tensorflow as tf 
import numpy as np 
import scipy.io
import time
tf.random.set_seed(1)
np.random.seed(1)
class LANet0(object):
    def __init__(self, *inputs, layers,batchsize):
        self.layers = layers
        self.num_layers = len(self.layers)

        if len(inputs) == 0:
            in_dim = self.layers[0]
            self.X_mean = np.zeros([1, in_dim])
            self.X_std = np.ones([1, in_dim])
        else:
            X = np.concatenate(inputs, 1)
            self.X_mean = X.mean(0, keepdims=True)
            self.X_std = X.std(0, keepdims=True)

        self.weights = []
        self.biases = []
        self.gammas = []
        self.bias_d = []
        for l in range(0,self.num_layers-1):
            in_dim = self.layers[l]
            out_dim = self.layers[l+1]
            W =  np.random.randn(in_dim,out_dim) * np.sqrt(2 / out_dim)
            #W = np.random.normal(size=[in_dim, out_dim])
            b = np.zeros([1, out_dim])
            g = np.ones([1, out_dim])
            # tensorflow variables
            self.weights.append(tf.Variable(W, dtype=tf.float64, trainable=True))
            self.biases.append(tf.Variable(b, dtype=tf.float64, trainable=True))
            self.gammas.append(tf.Variable(g, dtype=tf.float64, trainable=True))
        
        #定义一个ld的偏置
        bias_d = np.random.rand(batchsize,3)
        self.bias_d.append(tf.Variable(bias_d, dtype=tf.float64, trainable=True))

    @tf.function
    def __call__(self, *inputs):

        H = tf.concat(inputs, 1)
        #(tf.concat(inputs, 1) - self.X_mean)/self.X_std

        for l in range(0, self.num_layers-1):
            W = self.weights[l]
            b = self.biases[l]
            g = self.gammas[l]
            # weight normalization
            V = W/tf.norm(W, axis = 0, keepdims=True)
            # matrix multiplication
            H = tf.matmul(H, V)
            # add bias
            H = g*H + b
            # activation
            if l < self.num_layers-2:
                H = H*tf.sigmoid(H)

        Yo,Yd = tf.split(H, num_or_size_splits=2, axis=1)
        Yd = Yd+self.bias_d[0]
        return Yo,Yd

    @tf.function
    def get_l2_loss(self):
        W = self.weights
        l2_loss = 0
        for _,wi in enumerate(W):
            l2_loss = l2_loss+tf.norm(wi)**2
        return l2_loss

    def save_model(self,path=''):
        #保存模型参数
        W = self.weights
        b = self.biases
        g = self.gammas
        bd = self.bias_d

        W_dict = {}
        for i in range(len(W)):
            W_dict['w'+str(i)] = W[i].numpy()

        b_dict = {}
        for i in range(len(W)):
            b_dict['b'+str(i)] = b[i].numpy()
        
        g_dict = {}
        for i in range(len(W)):
            g_dict['g'+str(i)] = g[i].numpy()

        
        bd_dict = {}
        bd_dict['bd'] = bd[0].numpy()

        total_dict = {}
        total_dict.update(W_dict)
        total_dict.update(b_dict)
        total_dict.update(g_dict)
        total_dict.update(bd_dict)

        scipy.io.savemat('model_paras.mat' , total_dict)
        from google.colab import files
        files.download('model_paras.mat')

LAM = 0.01

class vehicle_net():
    def __init__(self,*inputs,layers,lo_dim,hp):
        self.epochs = hp["epochs"]
        self.tf_optimizer = tf.keras.optimizers.Adam(
            learning_rate=hp["tf_lr"],
            beta_1=hp["tf_b1"],
            epsilon=hp["tf_eps"])
        self.LAM = hp["lamda"]
        batchsize = hp['batchsize']
        self.model = LANet0(*inputs, layers=layers,batchsize=batchsize)
        
    
    @tf.function
    def cal_inverse_dynamic(self,q,dq,ddq):
        q = tf.convert_to_tensor(q)
        dq = tf.convert_to_tensor(dq)   
        dq = tf.reshape(dq,[dq.shape[0],dq.shape[1],1])  #NX3 -> NX3X1
        ddq = tf.convert_to_tensor(ddq)
        ddq = tf.reshape(ddq,[ddq.shape[0],ddq.shape[1],1])  #NX3 -> NX3X1
        sample_num = q.shape[0]
        with tf.GradientTape(persistent=True) as tp:
            tp.watch(q)

            Y_lo,Y_ld = self.model(q) 

            l1 = Y_ld[:,0:1]
            l2 = Y_ld[:,1:2]
            l3 = Y_ld[:,2:3]
            l4 = Y_lo[:,0:1]
            l5 = Y_lo[:,1:2]
            l6 = Y_lo[:,2:3]

            # #计算dLi_dq
            dL1_dq = tp.gradient(l1,q)  #NX3
            dL2_dq = tp.gradient(l2,q)  #NX3
            dL3_dq = tp.gradient(l3,q)  #NX3
            dL4_dq = tp.gradient(l4,q)  #NX3
            dL5_dq = tp.gradient(l5,q)  #NX3
            dL6_dq = tp.gradient(l6,q)  #NX3

            #计算dl_dqi
            dLd_dq1 = tf.stack([dL1_dq[:,0:1],dL2_dq[:,0:1],dL3_dq[:,0:1]],axis=1)  #NX3X1
            dLd_dq1 = tf.squeeze(dLd_dq1,axis=2)  #NX3
            dLd_dq2 = tf.stack([dL1_dq[:,1:2],dL2_dq[:,1:2],dL3_dq[:,1:2]],axis=1)
            dLd_dq2 = tf.squeeze(dLd_dq2,axis=2)  #NX3  
            dLd_dq3 = tf.stack([dL1_dq[:,2:3],dL2_dq[:,2:3],dL3_dq[:,2:3]],axis=1)
            dLd_dq3 = tf.squeeze(dLd_dq3,axis=2)  #NX3  

            dLo_dq1 = tf.stack([dL4_dq[:,0:1],dL5_dq[:,0:1],dL6_dq[:,0:1]],axis=1)
            dLo_dq1 = tf.squeeze(dLo_dq1,axis=2)  #NX3  
            dLo_dq2 = tf.stack([dL4_dq[:,1:2],dL5_dq[:,1:2],dL6_dq[:,1:2]],axis=1)  
            dLo_dq2 = tf.squeeze(dLo_dq2,axis=2)  #NX3
            dLo_dq3 = tf.stack([dL4_dq[:,2:3],dL5_dq[:,2:3],dL6_dq[:,2:3]],axis=1)  
            dLo_dq3 = tf.squeeze(dLo_dq3,axis=2)  #NX3
        del tp



        #计算逆动力学方程中的各项：M = J*ddq-0.5*[dq_T*(dl_dq1*L_T+L*dL_dq1_T)*dq ...]+dJ*dq
        #1.J = L*L'
        L_mat,L_mat_T,J = self.cal_J(Y_lo,Y_ld)

        #2.dJ/dt = dLdt@L'+L@dL'dt = dLdt@L'+L@dLdt'
        dL1_dq = tf.reshape(dL1_dq,[dL1_dq.shape[0],1,dL1_dq.shape[1]])
        dL2_dq = tf.reshape(dL2_dq,[dL2_dq.shape[0],1,dL2_dq.shape[1]])
        dL3_dq = tf.reshape(dL3_dq,[dL3_dq.shape[0],1,dL3_dq.shape[1]])
        dL4_dq = tf.reshape(dL4_dq,[dL4_dq.shape[0],1,dL4_dq.shape[1]])
        dL5_dq = tf.reshape(dL5_dq,[dL5_dq.shape[0],1,dL5_dq.shape[1]])
        dL6_dq = tf.reshape(dL6_dq,[dL6_dq.shape[0],1,dL6_dq.shape[1]])

        dl11 = tf.squeeze(dL1_dq@dq,axis=2)   #NX1X1 -> NX1
        dl22 = tf.squeeze(dL2_dq@dq,axis=2)
        dl33 = tf.squeeze(dL3_dq@dq,axis=2)

        dl21 = tf.squeeze(dL4_dq@dq,axis=2)
        dl31 = tf.squeeze(dL5_dq@dq,axis=2)
        dl32 = tf.squeeze(dL6_dq@dq,axis=2)

        dl12 = tf.zeros((dl11.shape[0],1),dtype = tf.float64)
        dl13 = tf.zeros((dl11.shape[0],1),dtype = tf.float64)
        dl23 = tf.zeros((dl11.shape[0],1),dtype = tf.float64)

        dl = tf.stack([dl11,dl12,dl13,dl21,dl22,dl23,dl31,dl32,dl33],axis=1)
        dl = tf.squeeze(dl,axis=2)
        dl_mat = tf.reshape(dl,[dl.shape[0],3,3])

        dl_mat_T = tf.transpose(dl_mat,perm=[0,2,1])  #将NX3X3 中第二维和第三维进行转置 

        dJ = dl_mat@L_mat_T+L_mat@dl_mat_T

        #3.(dq'@J@dq)/dq
        #dLo_dq1 = dY_lo[:,:,0]     #NX3
        #dLd_dq1 = dY_ld[:,:,0]
        dl_dq1_mat = self.vec2mat(dLd_dq1,dLo_dq1)   #dl_dq1

        # dLo_dq2 = dY_lo[:,:,1]
        # dLd_dq2 = dY_ld[:,:,1]
        dl_dq2_mat = self.vec2mat(dLd_dq2,dLo_dq2)

        # dLo_dq3 = dY_lo[:,:,2]
        # dLd_dq3 = dY_ld[:,:,2]
        dl_dq3_mat = self.vec2mat(dLd_dq3,dLo_dq3)

        #计算dq_T*(dl_dq1*L_T+L*dL_dq1_T)*dq
        dq_T = tf.transpose(dq,perm=[0,2,1])    #NX3X1 ->NX1X3
        tmp1 = dq_T@(dl_dq1_mat@L_mat_T+L_mat@tf.transpose(dl_dq1_mat,perm=[0,2,1]))@dq   #NX1X1
        tmp2 = dq_T@(dl_dq2_mat@L_mat_T+L_mat@tf.transpose(dl_dq2_mat,perm=[0,2,1]))@dq   #NX1X1
        tmp3 = dq_T@(dl_dq3_mat@L_mat_T+L_mat@tf.transpose(dl_dq3_mat,perm=[0,2,1]))@dq   #NX1X1
        tmp = tf.stack([tmp1,tmp2,tmp3],axis=1)
        tmp = tf.squeeze(tmp,axis=3)   #NX3X1X1  -> NX3X1
        #-0.5*[dq_T*(dl_dq1*L_T+L*dL_dq1_T)*dq ...]   #NX3X1
        f2 = -0.5*tmp

        #4.计算M = J*ddq-0.5*[dq_T*(dl_dq1*L_T+L*dL_dq1_T)*dq ...]+dJ*dq
        M_pre0 = J@ddq+f2+dJ@dq

        gama,phi = q[:,0:1],q[:,1:2]
        r11 = tf.ones_like(gama)
        r12 = tf.zeros_like(gama)
        r13 = -tf.sin(phi)
        r21 = tf.zeros_like(gama)
        r22 = tf.cos(gama)
        r23 = tf.sin(gama)*tf.cos(phi)
        r31 = tf.zeros_like(gama)
        r32 = -tf.sin(gama)
        r33 = tf.cos(gama)*tf.cos(phi)

        R = tf.stack([r11,r12,r13,r21,r22,r23,r31,r32,r33],axis=1)
        R = tf.squeeze(R,axis=2)
        R = tf.reshape(R,[R.shape[0],3,3])
        R_T = tf.transpose(R,perm=[0,2,1])  #将NX3X3 中第二维和第三维进行转置

        M_pre = tf.linalg.inv(R_T)@M_pre0


        return M_pre0    #NX3X1

    @tf.function
    def vec2mat(self,dLd_dq1,dLo_dq1):
        #将N个3X1的dld_dqi 以及dlo_dqi 组合成 N个 3X3的矩阵
        dl_dq11 = dLd_dq1[:,0:1]
        dl_dq12 = tf.zeros((dl_dq11.shape[0],1),dtype = tf.float64)
        dl_dq13 = tf.zeros((dl_dq11.shape[0],1),dtype = tf.float64)

        dl_dq21 = dLo_dq1[:,0:1]
        dl_dq22 = dLd_dq1[:,1:2]
        dl_dq23 = tf.zeros((dl_dq11.shape[0],1),dtype = tf.float64)

        dl_dq31 = dLo_dq1[:,1:2]
        dl_dq32 = dLo_dq1[:,2:3]
        dl_dq33 = dLd_dq1[:,2:3]

        dl = tf.stack([dl_dq11,dl_dq12,dl_dq13,dl_dq21,dl_dq22,dl_dq23,dl_dq31,dl_dq32,dl_dq33],axis=1)
        dl = tf.squeeze(dl,axis=2)
        dl_dq_mat = tf.reshape(dl,[dl.shape[0],3,3])

        return dl_dq_mat

    @tf.function  
    def cal_J(self,Lo,Ld):
       
        l1 = Ld[:,0:1]
        l2 = tf.zeros((l1.shape[0],1),dtype = tf.float64)
        l3 = tf.zeros((l1.shape[0],1),dtype = tf.float64)
        l4 = Lo[:,0:1]
        l5 = Ld[:,1:2]
        l6 = tf.zeros((l1.shape[0],1),dtype = tf.float64)
        l7 = Lo[:,1:2]
        l8 = Lo[:,2:3]
        l9 = Ld[:,2:3]

        L_vec = tf.stack([l1,l2,l3,l4,l5,l6,l7,l8,l9],axis=1)
        L_vec = tf.squeeze(L_vec,axis=2)
        L_mat = tf.reshape(L_vec,[L_vec.shape[0],3,3])

        L_vec_T = tf.stack([l1,l4,l7,l2,l5,l8,l3,l6,l9],axis=1)
        L_vec_T = tf.squeeze(L_vec_T,axis=2)
        L_mat_T = tf.reshape(L_vec_T,[L_vec_T.shape[0],3,3])

        J_mat = L_mat@L_mat_T

        #J_vec_total = tf.reshape(J_mat,[J_mat.shape[0],9])
        return L_mat,L_mat_T,J_mat  

    @tf.function
    def train(self,q,dq,ddq,mom0):

        with tf.GradientTape() as tp:

            M_pre0 = self.cal_inverse_dynamic(q,dq,ddq)
            res_loss = tf.reduce_mean(tf.keras.losses.MSE(mom0,M_pre0))
            loss = res_loss+LAM*self.model.get_l2_loss()

        variables = self.model.weights+self.model.biases+self.model.gammas+self.model.bias_d
        grads = tp.gradient(loss,variables)
        self.tf_optimizer.apply_gradients(grads_and_vars=zip(grads,variables))
        del tp
        #print(epoch,loss.numpy())
        return loss



    def fit(self,q,dq,ddq,Mom,mini_batch_size=128):     
        loss_save = []
        N_data = q.shape[0]
        for ep in range(self.epochs):

            #idx_data = np.random.choice(N_data, min(mini_batch_size, N_data))
            qi   = q#[idx_data,:]
            dqi  = dq#[idx_data,:]
            ddqi = ddq#[idx_data,:]
            Momi = Mom#[idx_data,:]
            Momi = tf.reshape(Momi,[Momi.shape[0],Momi.shape[1],1])
            time1 = time.time()                           
            loss = self.train(qi,dqi,ddqi,Momi)                
            loss_numpy = loss.numpy()
            loss_save.append(loss_numpy) 
            if ep%100==0:
                time2 = time.time()
                tt1 = time2-time1
                tf.print(tt1,ep,loss_numpy)
        return loss_save

def main():
    #1.数据准备
    state_roll = np.loadtxt('./f16flyingdatasets/state_roll2020-9-25.dat')
    state_pitch = np.loadtxt('./f16flyingdatasets/state_pitch2020-9-25.dat')
    state_yaw = np.loadtxt('./f16flyingdatasets/state_yaw2020-9-25.dat')
    state = np.vstack((state_roll,state_pitch,state_yaw))

    state = state

    q = state[:,0:3]
    dq = state[:,3:6]
    ddq = state[:,6:9]
    mom = state[:,9:12]
    #mom0 = tf.reshape(mom,[mom.shape[0],mom.shape[1],1])
    #2.网络初始化
    layers = [3]+9*[6]+[6]
    hp = {}
    hp["epochs"] = 200000
    hp["tf_lr"] = 0.01
    hp["tf_b1"] = 0.9
    hp["tf_eps"] = 0.000000001
    hp["lamda"] = 0.01
    hp['batchsize'] = q.shape[0]
    vnet = vehicle_net(q,layers=layers,lo_dim=3,hp=hp)
    
    #3.训练
    losses = vnet.fit(q,dq,ddq,mom,mini_batch_size=q.shape[0])

    #保存结果
    #保存结果
    np.savetxt('loss.dat',losses)
    path = ''
    vnet.model.save_model(path)


if __name__ == "__main__":
    main()


15.89063048362732 0 13674457840.543165
0.11554336547851562 100 183494733.27800378
0.11529064178466797 200 174072073.99492204
0.11522126197814941 300 173274743.73153257
0.1156153678894043 400 172766212.26824442
0.11541247367858887 500 172422407.5735899
0.11644244194030762 600 172174851.2285017
0.11563730239868164 700 171986098.34316278
0.11525273323059082 800 171834477.49334568
0.11581754684448242 900 171706329.28102785
0.11696481704711914 1000 171591942.02455503
0.11603116989135742 1100 171483292.85564622
0.1153409481048584 1200 171373032.6640887
0.11532735824584961 1300 171254064.19885147
0.1157524585723877 1400 171118426.00630105
0.11541938781738281 1500 170954878.56456542
0.11549043655395508 1600 170744274.602689
0.11548280715942383 1700 170448578.79347935
0.11574125289916992 1800 169980011.54496068
0.11573982238769531 1900 169112495.97815442
0.11572623252868652 2000 167019861.54004845
0.11596059799194336 2100 7222968.212797586
0.11553835868835449 2200 449025.1631458807
0.1156582832

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [11]:
from google.colab import files
files.download('loss.dat')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [1]:
!git clone https://github.com/cha0stooo/f16flyingdatasets.git


Cloning into 'f16flyingdatasets'...
remote: Enumerating objects: 17, done.
remote: Counting objects: 100% (17/17), done.
remote: Compressing objects: 100% (17/17), done.
remote: Total 17 (delta 6), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (17/17), done.


In [10]:
!ls -R


.:
f16flyingdatasets  loss.dat  model_paras.mat  sample_data

./f16flyingdatasets:
lagrange_net.ipynb	  state_pitch2020-9-25.dat  state_yaw2020-9-16.dat
myADNet.ipynb		  state_roll2020-9-16.dat   state_yaw2020-9-25.dat
state_pitch2020-9-16.dat  state_roll2020-9-25.dat

./sample_data:
anscombe.json		      mnist_test.csv
california_housing_test.csv   mnist_train_small.csv
california_housing_train.csv  README.md
